In [1]:
from notebook_log_loading import load_sensor_log_file

sensor_log,first_timestamp=load_sensor_log_file("../to_study/senw_imu.txt")

In [5]:
import sys, os
parent_path = os.path.normpath(os.path.abspath(os.path.join(os.getcwd(), os.path.pardir)))
#print(path2add)
sys.path.append(parent_path)
sys.path.append(os.path.normpath(os.path.join(parent_path,"gyrii")))
sys.path.append(os.path.normpath(os.path.join(parent_path,"gyrii","underpinnings")))

from gyrii.Gyrus import ThreadedGyrus
from gyrii.underpinnings.NNCalibration2 import StatePredictorWithPolicy


class VisualMotionCalibGyrus(ThreadedGyrus):
    def __init__(self,broker,load_file=None):

        self.motionpredictor=nn.Sequential(nn.Linear(4,4),
                              nn.Tanh(),
                              nn.Linear(4,2))

        self.policy=nn.Sequential(nn.Linear(4,4),
                                  nn.Tanh(),
                                  nn.Linear(4,2),
                                  nn.Tanh())
        
        self.last_gyro=np.zeros(3) #last gyroscope reading
        self.last_accel=np.zeros(3) #last accelerometer reading
        
        
    def get_keys(self):
        return []
    
    def get_name(self):
        return "VisualMotionCalibGyrus"
    
    def holding_still(self,timestamp): #return true if I think I'm holding still
        if np.linalg.norm(self.last_gyro)>0.03:
            return False
        if np.linalg.norm(self.last_accel)>0.15:
            return False
        
